## Assignment 6 : Dimensionality Reduction 
### explore matrix factorization collaborative filtering
#### object
* top movies for 2 features
* recommendations for a user
    * score $s_{ui} = \sum_f {a_{uf}}\sigma_{f}b_{if}$
    
#### data 
* item : feature values for 100 iems and 15 features & weight for each feature
* users : feature values for 20 users and 15 features

In [1]:
import numpy as np
import pandas as pd

In [2]:
import xlrd

In [4]:
user = pd.read_excel("data.xlsx",
                  sheet_name = 'Users',
                    header = 0)

In [28]:
item = pd.read_excel("data.xlsx",
                  sheet_name = 'Items')
feature_weight = item[item.index == 0].iloc[:,2:17]
item_feature = item.iloc[2:,:17]
item_feature.columns = ["movieID","title"] + list(range(1,16))

In [30]:
item_feature

,movieID,title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
2,11,Star Wars: Episode IV - A New Hope (1977),-0.106651,-0.094401,0.199949,0.025196,-0.064437,0.276259,-0.206928,0.054093,-0.025950,-0.088964,-0.117164,0.104474,0.049532,0.140639,-0.031206
3,12,Finding Nemo (2003),-0.023980,-0.026960,-0.038067,0.248296,-0.118894,-0.068092,0.064117,0.037390,-0.082457,0.037841,-0.067603,-0.017252,-0.069223,0.063107,0.159913
4,13,Forrest Gump (1994),-0.157042,-0.039889,-0.224400,0.062452,0.098026,0.078255,-0.047104,0.147743,-0.100540,-0.018727,0.076231,0.016129,-0.040507,-0.275025,-0.052564
5,14,American Beauty (1999),-0.044468,0.198715,-0.006577,0.008760,0.056093,-0.020370,-0.046150,0.007244,0.008791,-0.067562,0.063375,-0.177547,-0.100531,0.068353,-0.085437
6,22,Pirates of the Caribbean: The Curse of the Bla...,0.036256,-0.132421,0.036410,0.007510,-0.010657,-0.065372,0.092768,0.003816,-0.081900,0.083958,0.167740,-0.095340,0.175285,0.106542,-0.287540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,9806,The Incredibles (2004),0.011210,-0.004647,0.030692,0.187177,-0.211637,-0.091898,0.071544,-0.046873,-0.035990,0.052422,-0.101781,-0.019164,-0.079617,0.005675,0.127663
98,10020,Beauty and the Beast (1991),0.090143,0.000451,-0.015548,0.262151,-0.004626,-0.028204,-0.000892,0.051988,0.010581,0.037816,-0.103800,-0.061565,0.121976,0.108155,-0.061090
99,36657,X-Men (2000),0.056898,-0.081200,0.065979,-0.089944,-0.116017,-0.106970,-0.004191,-0.105029,-0.014137,-0.038457,-0.019622,-0.037228,0.078752,-0.168768,0.008502
100,36658,X2: X-Men United (2003),0.081364,-0.065050,0.070287,-0.069999,-0.077794,-0.110695,-0.029912,-0.091040,-0.028855,-0.063204,-0.035059,-0.029101,0.035655,-0.151550,-0.036138


### Quiz 1,2.  Movie - Feature Relevance

In [36]:
item_feature.sort_values(by=1,ascending = False)

,movieID,title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
88,4327,Charlie's Angels (2000),0.281990,0.080262,0.112268,-0.043805,0.022552,-0.093860,-0.066124,0.047216,0.044850,0.061068,-0.045692,-0.068778,-0.123570,0.098928,-0.281855
39,414,Batman Forever (1995),0.218089,-0.009324,0.050725,-0.073225,0.083830,-0.010205,-0.116477,-0.034974,-0.199162,-0.090010,-0.065392,-0.010013,0.042187,-0.094728,-0.245821
87,3049,Ace Ventura: Pet Detective (1994),0.190879,0.067747,0.002918,-0.033348,0.074343,0.285012,0.386962,-0.088550,-0.025470,-0.153609,0.019709,0.069423,-0.039800,0.044615,0.016156
92,8467,Dumb & Dumber (1994),0.190638,0.110683,-0.010021,-0.012114,0.138720,0.308335,0.394490,-0.145662,-0.015438,-0.157104,-0.121444,0.219125,-0.152166,0.034849,0.202725
70,854,The Mask (1994),0.158601,-0.006160,-0.020265,0.007000,-0.020494,0.228476,0.273633,-0.008621,-0.078120,-0.023222,0.197461,-0.101675,-0.063419,-0.131777,-0.116862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,122,The Lord of the Rings: The Return of the King ...,-0.159160,-0.229643,0.211321,0.034750,0.291393,-0.039639,0.127626,-0.105860,0.157103,0.155752,-0.054644,-0.075175,-0.078339,-0.021426,-0.039500
61,680,Pulp Fiction (1994),-0.166983,0.189565,0.067380,-0.085505,-0.006725,0.035841,0.069947,0.121598,-0.004379,-0.085352,0.049630,-0.004048,0.007686,0.028500,-0.230013
29,238,The Godfather (1972),-0.183012,0.000505,-0.104087,0.015340,0.025326,0.029291,0.067782,0.067595,-0.013022,-0.174143,-0.045478,0.036745,-0.072569,0.049888,-0.284437
35,278,The Shawshank Redemption (1994),-0.194844,-0.008067,-0.179663,-0.016005,0.035038,-0.012615,0.031494,-0.014781,0.003321,-0.094743,-0.027566,0.071182,0.023811,-0.121597,-0.097829


In [37]:
item_feature.sort_values(by=2,ascending = False)

,movieID,title,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
5,14,American Beauty (1999),-0.044468,0.198715,-0.006577,0.008760,0.056093,-0.020370,-0.046150,0.007244,0.008791,-0.067562,0.063375,-0.177547,-0.100531,0.068353,-0.085437
61,680,Pulp Fiction (1994),-0.166983,0.189565,0.067380,-0.085505,-0.006725,0.035841,0.069947,0.121598,-0.004379,-0.085352,0.049630,-0.004048,0.007686,0.028500,-0.230013
7,24,Kill Bill: Vol. 1 (2003),-0.045203,0.181570,0.205705,-0.171394,-0.005189,-0.115007,0.180589,0.525397,0.031987,0.047834,-0.138115,0.032692,0.088862,-0.048018,0.130020
34,275,Fargo (1996),-0.043682,0.161559,0.033419,0.022273,-0.009139,-0.015173,-0.073476,-0.041835,0.073475,-0.052947,-0.014390,0.021147,-0.101912,0.029872,0.000430
8,38,Eternal Sunshine of the Spotless Mind (2004),-0.048743,0.161058,0.014168,0.024869,0.047061,-0.040450,-0.020911,-0.109095,0.027812,-0.055463,0.059242,-0.126222,-0.022280,0.059654,0.029783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,1894,Star Wars: Episode II - Attack of the Clones (...,0.103738,-0.155550,0.111279,-0.104832,0.167445,0.182787,-0.223961,0.078515,-0.384900,-0.088922,-0.073515,0.039891,0.076949,0.028238,0.070819
53,604,The Matrix Reloaded (2003),0.035279,-0.182207,0.000107,-0.229257,0.095650,-0.020731,-0.026350,0.064681,-0.404386,-0.036515,0.108601,-0.249963,-0.232427,0.093184,0.266010
16,120,The Lord of the Rings: The Fellowship of the R...,-0.157498,-0.228167,0.227916,0.038214,0.290180,-0.038963,0.126902,-0.085299,0.167285,0.157715,-0.026913,-0.060022,-0.105362,-0.020912,-0.034652
17,121,The Lord of the Rings: The Two Towers (2002),-0.141333,-0.228428,0.226529,0.035853,0.298140,-0.041196,0.121425,-0.080461,0.137022,0.157701,-0.039055,-0.075556,-0.072986,-0.041188,-0.036740


### Quiz 3.  Recommendation

In [70]:
p_f =  user.loc[user.User == 4469,1:]
p_f

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
8,-0.002139,-0.001049,-0.004224,-0.001461,0.003396,0.001352,-0.000004,-0.001165,-0.000785,-0.002333,-0.001737,0.001324,0.002993,0.000662,-0.001001


In [44]:
feature_weight

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,203.615022,121.021257,102.797882,96.830823,85.133738,82.763808,79.835,77.66642,74.607018,73.10941,69.09656,68.136588,66.480305,64.513088,63.569436


In [61]:
float(feature_weight[1]) * item_feature[1]

2     -21.715813
3      -4.882766
4     -31.976067
5      -9.054366
6       7.382226
         ...    
97      2.282623
98     18.354521
99     11.585314
100    16.566832
101    24.743212
Name: 1, Length: 100, dtype: float64

In [72]:
weighted_item_feature = item_feature.set_index(["movieID","title"])
for i in range(1,16):
    weighted_item_feature[i] = weighted_item_feature [i] * float(feature_weight[i])
weighted_item_feature

,,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
movieID,title,,,,,,,,,,,,,,,
11,Star Wars: Episode IV - A New Hope (1977),-21.715813,-11.424585,20.554339,2.439722,-5.485769,22.864286,-16.520092,4.201176,-1.936072,-6.504127,-8.095605,7.118496,3.292900,9.073080,-1.983740
12,Finding Nemo (2003),-4.882766,-3.262738,-3.913246,24.042696,-10.121929,-5.635559,5.118772,2.903980,-6.151891,2.766519,-4.671159,-1.175484,-4.601981,4.071230,10.165575
13,Forrest Gump (1994),-31.976067,-4.827393,-23.067861,6.047319,8.345353,6.476658,-3.760545,11.474657,-7.500995,-1.369106,5.267317,1.098988,-2.692918,-17.742731,-3.341438
14,American Beauty (1999),-9.054366,24.048763,-0.676076,0.848200,4.775400,-1.685928,-3.684385,0.562623,0.655904,-4.939413,4.379018,-12.097423,-6.683317,4.409654,-5.431168
22,Pirates of the Caribbean: The Curse of the Black Pearl (2003),7.382226,-16.025797,3.742896,0.727223,-0.907291,-5.410461,7.406146,0.296338,-6.110314,6.138135,11.590246,-6.496168,11.652975,6.873359,-18.278730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9806,The Incredibles (2004),2.282623,-0.562399,3.155067,18.124460,-18.017490,-7.605855,5.711747,-3.640429,-2.685136,3.832509,-7.032688,-1.305790,-5.292985,0.366097,8.115451
10020,Beauty and the Beast (1991),18.354521,0.054534,-1.598300,25.384344,-0.393863,-2.334241,-0.071222,4.037728,0.789422,2.764676,-7.172242,-4.194810,8.108986,6.977402,-3.883475
36657,X-Men (2000),11.585314,-9.826946,6.782471,-8.709365,-9.876984,-8.853233,-0.334607,-8.157257,-1.054749,-2.811563,-1.355787,-2.536600,5.235433,-10.887719,0.540495


In [76]:
pd.DataFrame(columns =  ["movieID","title","score"])

,movieID,title,score


In [83]:
]],ignore_index =True)

,0,1,2,movieID,score,title
0,1.0,2.0,3.0,NaN,NaN,NaN


In [89]:
user_4469_prf = pd.DataFrame(columns =  ["movieID","title","score"])
for idx,row in weighted_item_feature.iterrows():
    user_4469_prf = user_4469_prf.append({"movieID": idx[0],
                                          "title" : idx[1],
                                          "score" : float(np.dot(p_f,row))},
                                        ignore_index=True)
user_4469_prf.sort_values(by="score",ascending = False)             

,movieID,title,score
33,278,The Shawshank Redemption (1994),0.207680
39,453,A Beautiful Mind (2001),0.183286
10,98,Gladiator (2000),0.173611
27,238,The Godfather (1972),0.172180
2,13,Forrest Gump (1994),0.170744
...,...,...,...
46,585,Monsters Inc. (2001),-0.114932
43,558,Spider-Man 2 (2004),-0.122341
95,9806,The Incredibles (2004),-0.131435
52,607,Men in Black (a.k.a. MIB) (1997),-0.142724


In [80]:
for idx,row in weighted_item_feature.iterrows():
    print("title:",idx[1])
    print(np.dot(p_f,row))

title: Star Wars: Episode IV - A New Hope (1977)
[0.03354001]
title: Finding Nemo (2003)
[-0.06645858]
title: Forrest Gump (1994)
[0.17074429]
title: American Beauty (1999)
[-0.01518245]
title: Pirates of the Caribbean: The Curse of the Black Pearl (2003)
[-0.00715431]
title: Kill Bill: Vol. 1 (2003)
[-0.11352611]
title: Eternal Sunshine of the Spotless Mind (2004)
[-0.00436079]
title: Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
[-0.03287799]
title: Memento (2000)
[0.03411018]
title: Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
[-0.05485039]
title: Gladiator (2000)
[0.17361148]
title: Back to the Future (1985)
[-0.06035974]
title: Snatch (2000)
[0.0192697]
title: Pretty Woman (1990)
[0.01093566]
title: The Lord of the Rings: The Fellowship of the Ring (2001)
[0.02189263]
title: The Lord of the Rings: The Two Towers (2002)
[0.02497507]
title: The Lord of the Rings: The Return of the King (2003)
[0.04118859]
title: O Brother Where Art Thou? (2000)
[-0.0385